In [60]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import json

In [61]:
NER = ["EFFECT"]
set = "test"

In [62]:
l = []
for ner in NER:
	test_sentences = pd.read_json(f"../NER/{ner}/test.json", lines=True)
	test_output = pd.read_json(f"../NER_output/{ner}/test_outputs.json")
	merged = pd.merge(test_sentences, test_output, left_index=True, right_index=True)
	merged.loc[:,"ner"] = ner
	l.append(merged)

df = pd.concat(l)

In [63]:
df = df.reset_index().drop(columns=["index"])

In [64]:
df

,tokens,ner_tags,id,predictions,label_ids,word_ids,ner
0,"[Taking, into, consideration, the, above, evid...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",0,"[[-1.8748829364776611, -1.967904806137085, 4.0...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...",EFFECT
1,"[The, two, draft, genomes, (, Nitrospira, sp.,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",1,"[[-2.734492301940918, -2.349016666412353, 5.28...","[-100, 2, 2, 2, 2, 2, 2, -100, -100, 2, -100, ...","[None, 0, 1, 2, 3, 4, 5, 5, 5, 6, 6, 7, 8, 8, ...",EFFECT
2,"[Sle1, is, a, substrate, of, the, ClpXP, prote...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",2,"[[-1.911755681037902, -1.535628080368042, 3.48...","[-100, 2, -100, 2, 2, 2, 2, 2, 2, -100, -100, ...","[None, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, 9, ...",EFFECT
3,"[In, this, study, ,, we, sequenced, and, assem...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",3,"[[-2.416978597640991, -2.321799755096435, 5.10...","[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100, -10...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 10,...",EFFECT
4,"[MF59, is, an, oil-in-water, emulsion, with, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",4,"[[-2.241502523422241, -2.455163955688476, 4.54...","[-100, 2, -100, 2, 2, 2, -100, -100, -100, -10...","[None, 0, 0, 1, 2, 3, 3, 3, 3, 3, 4, 5, 6, 7, ...",EFFECT
...,...,...,...,...,...,...,...
411,"[For, the, housekeeping, gene, GAPDH, ,, 20, s...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",411,"[[-2.416497945785522, -1.909170269966125, 4.49...","[-100, 2, 2, 2, -100, -100, 2, 2, 2, 2, 2, -10...","[None, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 7, 7, 8, ...",EFFECT
412,"[Chemotaxonomic, analyses, showed, that, type,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",412,"[[-2.4317431449890132, -2.126706838607788, 4.5...","[-100, 2, -100, -100, 2, 2, 2, 2, 2, 2, -100, ...","[None, 0, 0, 0, 1, 2, 3, 4, 5, 6, 6, 6, 7, 8, ...",EFFECT
413,"[[, 6, ], Some, volatile, sesquiterpenoids, ar...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",413,"[[-2.476452589035034, -1.915351033210754, 3.87...","[-100, 2, 2, 2, 2, 2, 2, -100, 2, 2, 2, 2, 2, ...","[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 10...",EFFECT
414,"[Nanosynbacter, lyticus, stain, TM7x, of, the,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",414,"[[-2.470869064331054, -2.453198671340942, 5.44...","[-100, 2, -100, -100, -100, -100, 2, -100, 2, ...","[None, 0, 0, 0, 0, 0, 1, 1, 2, 3, 3, 3, 4, 5, ...",EFFECT


In [65]:
label_ids = df[["label_ids"]]

In [66]:
annotations = []
for j in range(len(df)):
	for i in zip(df.loc[j,"tokens"],df.loc[j,"ner_tags"]):
		if i[1] != "O":
			annotations.append({"label":i[1],"text":i[0]})

### Predict sentences again

In [85]:
ner = "DISEASE"

In [86]:
path= f"../NER_output/{ner}"

tokenizer = AutoTokenizer.from_pretrained(path, model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(path)

nlp = pipeline(task='ner', model=model, tokenizer=tokenizer,
               aggregation_strategy="average", device=1)


In [87]:
import jsonlines

# Specify the path to the JSONL file
jsonl_file = f"../NER/{ner}/test.jsonls"

predictions = []
# Open the JSONL file
with jsonlines.open(jsonl_file) as reader:
    # Iterate over each line in the file
    for line in reader:
        for l in line:

            # Get the value from the "text" field
            text = l["data"]["text"]
            # Pass the value to the nlp() function
            print(text)
            predict = nlp(text)
            predictions.append(predict)
            print(predict)

Colony formation was inhibited by 64% and 79% in the presence of 25 µM N4A and YN1, respectively.
[]
Salmonella enterica (S. enterica) ATCC 14028, Escherichia coli (E. coli) ATCC 25992, Pseudomonas aeruginosa (P. aeruginosa) ATCC 13388, Staphylococcus aureus (S. aureus) ATCC 25923 and Enterococcus faecalis (E. faecalis) ATCC 29,122 were stored in cryovials at −80 °C until use.
[]
The pathway of dTDP-beta-L-rhamnose biosynthesis occurred in the P. distincta strains U2A, SM1926, and AC163.
[]
The need for an animal model to understand human gastritis and cancer is strongly reflected in the literature.
[{'entity_group': 'B', 'score': 0.64154977, 'word': 'gastritis', 'start': 49, 'end': 58}]
Strain AB26 uses the glutamine synthetase and glutamate synthase for ammonia assimilation in the cell.
[]
Here, we report the complete genome sequence of Mucilaginibacter strain 21P, isolated from an estuarine soil contaminated with iron mine tailings from the Samarco disaster, which occurred in 2015 i

In [88]:
predictions = [p for p in predictions if p]


In [89]:
predictions

[[{'entity_group': 'B',
   'score': 0.64154977,
   'word': 'gastritis',
   'start': 49,
   'end': 58}],
 [{'entity_group': 'B',
   'score': 0.6235176,
   'word': 'uc',
   'start': 502,
   'end': 504}],
 [{'entity_group': 'B',
   'score': 0.53988737,
   'word': 'cholera',
   'start': 406,
   'end': 413}],
 [{'entity_group': 'B',
   'score': 0.811898,
   'word': 'lung',
   'start': 313,
   'end': 317},
  {'entity_group': 'I',
   'score': 0.5876844,
   'word': 'cancer',
   'start': 318,
   'end': 324}],
 [{'entity_group': 'B',
   'score': 0.8681262,
   'word': 'melanoma',
   'start': 80,
   'end': 88}],
 [{'entity_group': 'B',
   'score': 0.7217178,
   'word': 'breast',
   'start': 137,
   'end': 143},
  {'entity_group': 'I',
   'score': 0.82631975,
   'word': 'cancer',
   'start': 144,
   'end': 150}],
 [{'entity_group': 'B',
   'score': 0.40122947,
   'word': 'grapevine',
   'start': 50,
   'end': 59},
  {'entity_group': 'I',
   'score': 0.514108,
   'word': 'anthracnose',
   'start': 6

In [90]:
annotations

[{'label': 'B', 'text': 'activity'},
 {'label': 'B', 'text': 'β-lactam'},
 {'label': 'I', 'text': 'hyper-resistance'},
 {'label': 'B', 'text': 'resistance'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': 'extreme'},
 {'label': 'I', 'text': 'iron'},
 {'label': 'I', 'text': 'concentrations'},
 {'label': 'B', 'text': 'ISR'},
 {'label': 'B', 'text': 'neuroprotective'},
 {'label': 'I', 'text': 'effects'},
 {'label': 'B', 'text': 'neuronal'},
 {'label': 'I', 'text': 'health'},
 {'label': 'B', 'text': 'lateral'},
 {'label': 'I', 'text': 'root'},
 {'label': 'I', 'text': 'formation'},
 {'label': 'B', 'text': 'auxin'},
 {'label': 'I', 'text': 'biosynthesis'},
 {'label': 'B', 'text': 'confers'},
 {'label': 'I', 'text': 'resistance'},
 {'label': 'I', 'text': 'to'},
 {'label': 'I', 'text': 'Pb'},
 {'label': 'I', 'text': '('},
 {'label': 'I', 'text': 'II'},
 {'label': 'I', 'text': ')'},
 {'label': 'I', 'text': 'salts'},
 {'label': 'B', 'text': 'non-adherent'},
 {'label': 'I', 'text': 'growi

In [94]:
for prediction in predictions:
    pred = prediction[0]["word"]
    count = 0
    for annotation in annotations:
        ann = annotation["text"]
        if ann in pred:
            count +=1
    if count <5:
        print(pred)

gastritis
uc
cholera
lung
melanoma
breast
fibrosarcoma
rrms
stem
axial
septicemia
asthma


In [83]:
ann

'microbiome'